# load iridium data and process

In [1]:
from glob import glob

import pandas as pd
import xarray as xr

# cognac data and tools
import cognac.insitu as iu

---
## load and select data

In [2]:
# data directory and filename
#
#campaign = 'microco_leg2.yaml'
#campaign = 'lopstech19_leg1.yaml'
campaign = 'cognac20.yaml'

cp = iu.campaign(campaign)


#data_dir = cp.path+'iridium/'
#imeis={unit: cp[unit]['iridium'] for unit in ['source', 'enregistreur']}
#imeis={unit: cp[unit]['iridium'] for unit in ['source', 'enregistreur']}

## utils

In [3]:
unit = 'emetteur'
imei = cp[unit]['instruments']['iridium']

In [4]:
def load_iridium(unit):
    
    if 'instruments' in unit and 'iridium' in unit['instruments']:
        
        imei = unit['instruments']['iridium']
        
        dummy_path = 'home1/triagoz/intranet/htdocs/LOPStrack/DATA/'
        imei_file = cp.path+'iridium/'+dummy_path+'{}/synthese_{}.txt'.format(imei, imei)
        
        df = pd.read_csv(imei_file,
                        sep='\t+',
                        parse_dates=[0],
                        infer_datetime_format=True,
                        index_col=False,
                        names=['time','lat','lon','latlon0']
                         )
        df = df.set_index('time').sort_index()
        
        # select only time during deployments
        #t_min = min([d.start.time for d in u])
        #t_max = max([d.end.time for d in u])
        #df = df[t_min:t_max]
        
        # transform into a gps instance
        _gps = iu.gps.gps(lon=df.lon, lat=df.lat, time=df.index)
        
        # trim and divide to keep deployments time
        gps = {d.label: _gps.clean(d, inplace=False) for d in unit}
                    
        return gps

gps = load_iridium(cp[unit])

---
## process all units

In [14]:
# load and store data for all units
for unit in cp:
    gps = load_iridium(cp[unit])
    # store
    if gps:
        for label, lgps in gps.items():
            if lgps:
                lgps.to_nc(cp.pathp+unit+'_iridium_'+label+'.nc')

Data store to /Users/aponte/Current_projects/cognac/campagnes/2020_cognac20/data/datap/emetteur_iridium_j1.nc
Data store to /Users/aponte/Current_projects/cognac/campagnes/2020_cognac20/data/datap/emetteur_iridium_j2.nc
Data store to /Users/aponte/Current_projects/cognac/campagnes/2020_cognac20/data/datap/recepteur0_iridium_j1.nc
Data store to /Users/aponte/Current_projects/cognac/campagnes/2020_cognac20/data/datap/recepteur0_iridium_j2.nc
Data store to /Users/aponte/Current_projects/cognac/campagnes/2020_cognac20/data/datap/recepteur1_iridium_j1.nc
Data store to /Users/aponte/Current_projects/cognac/campagnes/2020_cognac20/data/datap/recepteur1_iridium_j2.nc
Data store to /Users/aponte/Current_projects/cognac/campagnes/2020_cognac20/data/datap/recepteur2_iridium_j1.nc
Data store to /Users/aponte/Current_projects/cognac/campagnes/2020_cognac20/data/datap/recepteur2_iridium_j2.nc


---
## reload units and plot trajectories

In [32]:
iridium_files = glob(cp.pathp+'*_iridium_*.nc')
units = set(f.split('/')[-1].split('_')[0] for f in iridium_files)

iridium = {}
for u in units:
    deployments = [f.split('/')[-1].split('_')[2].split('.')[0] for f in glob(cp.pathp+u+'_iridium_*.nc')]
    iridium[u] = {d: iu.gps.gps(file=cp.pathp+u+'_iridium_'+d+'.nc') for d in deployments}

iridium

{'emetteur': {'j2': <cognac.insitu.gps.gps at 0x13e46c940>,
  'j1': <cognac.insitu.gps.gps at 0x13e46c820>},
 'recepteur1': {'j2': <cognac.insitu.gps.gps at 0x13e46c0d0>,
  'j1': <cognac.insitu.gps.gps at 0x13ec5b610>},
 'recepteur2': {'j2': <cognac.insitu.gps.gps at 0x102e773a0>,
  'j1': <cognac.insitu.gps.gps at 0x13ebdca90>},
 'recepteur0': {'j1': <cognac.insitu.gps.gps at 0x13ebdc940>,
  'j2': <cognac.insitu.gps.gps at 0x13ebdca30>}}